In [4]:
import numpy as np
import RotRep
from GBdist import spv4dist, convertB
from functions import AAtoG, diffMisorientations, AnglestoV
import sys

# --------------------------------------------------------------
# writing the symmetry operators for the 4*4 matrix
O=RotRep.GetSymRotMat(symtype='Cubic')

bsym=np.zeros((24,4,4))
for i in range(24):
    bsym[i][:3,:3]=O[i]
    bsym[i][3,3]=1
    
# --------------------------------------------------------------
# the GBs of interest
seed1_angle = 22.9
seed1_axis = [-4,16,-5]
seed1 = AAtoG(seed1_angle, seed1_axis)
seed2_angle = 38.3
seed2_axis = [-7,6,1]
seed2 = AAtoG(seed2_angle, seed2_axis)
seed3_angle = 51.6
seed3_axis = [-2,6,9]
seed3 = AAtoG(seed3_angle, seed3_axis)

dg_seed1_1 = AAtoG(28.6, [23,5,4])   #080617_2_5
dg_seed1_2 = AAtoG(21.2, [4,-22,-11])   #110317_3
dg_seed2_1 = AAtoG(38.3, [-7,6,1])   #102716_5
dg_seed2_2 = AAtoG(38.7, [22,0,-21])   #081717_3
dg_seed2_3 = AAtoG(40.2, [-24,1,-19])   #110917_1_2
dg_seed3_1 = AAtoG(48.8, [-5,15,21])   #110917_2_1
dg_seed3_2 = AAtoG(48.2, [-1,-3,4])   #110917_2_2

# --------------------------------------------------------------
# generate a list of all possible normal directions
res_theta = 10
res_psi = 10
iter_theta = 360/res_theta + 1
iter_psi = 90/res_psi + 1
iter_total = iter_theta*iter_psi
norm_all = np.zeros((iter_total, 3))
norms = np.zeros((iter_total, 2))
for i in range(iter_psi):
    for j in range(iter_theta):
        theta = np.float(res_theta*j)
        psi = np.float(res_psi*i)
        norms[j + i*iter_theta] = [theta, psi]
        norm_all[j + i*iter_theta] = AnglestoV(theta, psi)


# --------------------------------------------------------------
# get the max distance between two misorientations with plane normals being all possible values
def get_b_all(dg, norm_all):
    b_all = np.zeros((iter_total, 4, 4))
    for i in range(iter_total):
        n = norm_all[i]
        b_all[i] = convertB(dg,n)
    return b_all
        

def max_dis(dg1, dg2):
    b1_all = get_b_all(dg1, norm_all)
    b2_all = get_b_all(dg2, norm_all)
    maxDist = 0.0
    for i in range(iter_total):
        sys.stdout.write('\r'+'iteration i: {0} '.format(i))
        for j in range(iter_total):
#             sys.stdout.write('\r'+'iteration j: {0} '.format(j))
            (v4dist,bb) = spv4dist(b1_all[i,:,:], b2_all[j,:,:], moreinfo=False)
            if v4dist > maxDist:
                maxDist = v4dist
    return maxDist


maxDist = max_dis(dg_seed1_1, dg_seed1_2)
print maxDist
# b1_all = get_b_all(dg_seed1_1, norm_all)


iteration i: 11 

KeyboardInterrupt: 

[[  0.00000000e+00   0.00000000e+00   1.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00]
 ..., 
 [  9.39692621e-01  -3.42020143e-01   6.12323400e-17]
 [  9.84807753e-01  -1.73648178e-01   6.12323400e-17]
 [  1.00000000e+00  -2.44929360e-16   6.12323400e-17]]


In [3]:
print norms

[[   0.    0.]
 [  10.    0.]
 [  20.    0.]
 [  30.    0.]
 [  40.    0.]
 [  50.    0.]
 [  60.    0.]
 [  70.    0.]
 [  80.    0.]
 [  90.    0.]
 [ 100.    0.]
 [ 110.    0.]
 [ 120.    0.]
 [ 130.    0.]
 [ 140.    0.]
 [ 150.    0.]
 [ 160.    0.]
 [ 170.    0.]
 [ 180.    0.]
 [ 190.    0.]
 [ 200.    0.]
 [ 210.    0.]
 [ 220.    0.]
 [ 230.    0.]
 [ 240.    0.]
 [ 250.    0.]
 [ 260.    0.]
 [ 270.    0.]
 [ 280.    0.]
 [ 290.    0.]
 [ 300.    0.]
 [ 310.    0.]
 [ 320.    0.]
 [ 330.    0.]
 [ 340.    0.]
 [ 350.    0.]
 [ 360.    0.]
 [   0.   10.]
 [  10.   10.]
 [  20.   10.]
 [  30.   10.]
 [  40.   10.]
 [  50.   10.]
 [  60.   10.]
 [  70.   10.]
 [  80.   10.]
 [  90.   10.]
 [ 100.   10.]
 [ 110.   10.]
 [ 120.   10.]
 [ 130.   10.]
 [ 140.   10.]
 [ 150.   10.]
 [ 160.   10.]
 [ 170.   10.]
 [ 180.   10.]
 [ 190.   10.]
 [ 200.   10.]
 [ 210.   10.]
 [ 220.   10.]
 [ 230.   10.]
 [ 240.   10.]
 [ 250.   10.]
 [ 260.   10.]
 [ 270.   10.]
 [ 280.   10.]
 [ 290.   

# Validations
- Moraweic's GBdist functions, the misorientations doesn't need to be in FZ first

In [ ]:
# --------------------------------------------------------------
# Test for the two misorientations don't need to be in FZ
dg1 = AAtoG(60, [1,1,1])
dg2 = AAtoG(60, [1,1,1])
print diffMisorientations(dg1, dg2)

b1 = convertB(dg1, np.array([1,0,0]))
b2 = convertB(dg2, np.array([1,0,0]))
(v4dist,bb) = spv4dist(b1,b2,moreinfo=False)
print v4dist